# Development of regression model to predict car prices

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
import pickle

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size ch

In [2]:
df_cars = pd.read_csv('cars.csv',sep=';',index_col=0)

In [3]:
df_cars.describe()

,Miltal,Modellår,Tillverkningsår,price
count,596.000000,596.000000,596.000000,596.000000
mean,8600.657718,2012.521812,2012.315436,92921.568792
std,7923.656805,5.181207,5.137226,60749.086953
min,0.000000,1997.000000,1997.000000,2500.000000
25%,1.000000,2010.000000,2010.000000,42125.000000
50%,7530.000000,2014.000000,2013.000000,83400.000000
75%,14103.250000,2017.000000,2017.000000,154925.000000
max,32499.500000,2018.000000,2018.000000,292500.000000


In [4]:
linear_regression = LinearRegression()

In [5]:
y = df_cars['price']
feature_colums = ['Miltal','Modellår','Tillverkningsår']
X = df_cars[feature_colums]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [7]:
degree = 2
model = make_pipeline(PolynomialFeatures(degree), linear_regression)

r2s = []
for i in range(20):

    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    r2 = r2_score(y_test, y_predict)
    r2s.append(r2)
np.mean(r2s)

/Users/martinalexandersson/Dev/blocket/venv/lib/python3.5/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


0.8963037901860675

In [8]:
model.fit(X, y)

Pipeline(memory=None,
     steps=[('polynomialfeatures', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('linearregression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [9]:
pickle.dump(model, open('model.pkl', 'wb'))

In [10]:
df_cars['predicted price'] = model.predict(df_cars[feature_colums])
df_cars['price rating'] = df_cars['price'] / df_cars['predicted price']

In [11]:
df_cars.sort_values(by = 'price rating')

,Bränsle,Miltal,Modellår,Tillverkningsår,Växellåda,header,price,car type,predicted price,price rating
80911462,Diesel,19249.5,2003.0,2001.0,Manuell,Berlingo -03,5000.0,berlingo,-755.607709,-6.617190
80733755,Diesel,2249.5,2017.0,2016.0,Manuell,Citroën Berlingo 4X4 dangel -17,4125.0,berlingo,146844.673263,0.028091
80836337,Diesel,32499.5,1998.0,1998.0,Manuell,"Reservdelsbil Berlingo skåp 1,8 deisel -98",3500.0,berlingo,21562.433124,0.162319
80426876,Bensin,11249.5,2004.0,2004.0,Manuell,Citroën Berlingo Family -04,5000.0,berlingo,25124.771837,0.199007
79977136,Diesel,32499.5,2000.0,2000.0,Manuell,Berlingo -00,4000.0,berlingo,18928.264526,0.211324
80613134,Bensin,18249.5,2004.0,2004.0,Manuell,Citroen berlingo 04 -04,3000.0,berlingo,12851.211616,0.233441
80410345,Diesel,26249.5,1997.0,1997.0,Manuell,Citroën Berlingo -97,3000.0,berlingo,12220.493577,0.245489
80818452,Diesel,32499.5,2001.0,2001.0,Manuell,"Citroen Berlingo 1,9 Diesel -01",5000.0,berlingo,18399.681178,0.271744
80603592,Bensin,32499.5,2001.0,2001.0,Manuell,Citroën Berlingo fa 1.6 -01,5000.0,berlingo,18399.681178,0.271744
80887193,Bensin,24249.5,2003.0,2003.0,Manuell,Citroen berlingo -03,2500.0,berlingo,8981.124499,0.278362


In [12]:
index = df_cars['header'] == 'Renault Kangoo 1,5 (6 växlad) -08'
df_cars.loc[index]

,Bränsle,Miltal,Modellår,Tillverkningsår,Växellåda,header,price,car type,predicted price,price rating
80790572,Diesel,15749.5,2008.0,2008.0,Manuell,"Renault Kangoo 1,5 (6 växlad) -08",45000.0,kangoo,30125.635766,1.493744


In [13]:
df_cars['car type'].unique()

array(['kangoo', 'berlingo'], dtype=object)

In [33]:
degree = 2
kangoo_model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
index = df_cars['car type'] == 'kangoo'
df_kangoo = df_cars.loc[index].copy()

kangoo_model.fit(df_kangoo[feature_colums],df_kangoo['price'])

Pipeline(memory=None,
     steps=[('polynomialfeatures', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('linearregression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [34]:
df_kangoo['predicted price'] = model.predict(df_kangoo[feature_colums])
df_kangoo['price rating'] = df_kangoo['price'] / df_kangoo['predicted price']
df_kangoo.sort_values(by = 'price rating')

,Bränsle,Miltal,Modellår,Tillverkningsår,Växellåda,header,price,car type,predicted price,price rating
80900137,Bensin,14723.0,2006.0,2006.0,Manuell,Renault Kangoo 1.6 95hk -06,14800.0,kangoo,23868.334848,0.620068
80818741,Bensin,17551.0,2006.0,2006.0,Manuell,Renault Kangoo 1.6 M-VÄRM 5-SITS DRAG -06,14900.0,kangoo,19350.643308,0.770000
80651074,Bensin,10249.5,2011.0,2011.0,Manuell,Renault Kangoo Bensin/E85 -11,48000.0,kangoo,60554.583967,0.792673
80696920,El,1441.0,2013.0,2013.0,Automat,Renault Kangoo Express Maxi II Z.E. Aut -13,89900.0,kangoo,112221.836521,0.801092
77656999,Diesel,12000.0,2009.0,2009.0,Manuell,Renault Kangoo 1.5 DCI Skåp / Nybesiktigad -09,34900.0,kangoo,43526.848853,0.801804
80653137,Diesel,630.0,2014.0,2013.0,Manuell,Renault Kangoo 1.5 dCi 90hk Style Kombi -14,109900.0,kangoo,129176.888572,0.850771
80603429,Bensin,6800.0,2009.0,2008.0,Manuell,Renault Kangoo 1.6|5-Sits|6800Mil|Fullservad -09,54900.0,kangoo,62411.169774,0.879650
80783111,Bensin,17249.5,2000.0,2000.0,Manuell,"RENAULT Kangoo 1,4 SNABB AFFÄR 8000 KR -00",8000.0,kangoo,8710.436678,0.918438
80896861,Diesel,2100.0,2014.0,2014.0,Manuell,Renault Kangoo II Kombi phII dCi 90FAP S/S St -14,109500.0,kangoo,117250.053527,0.933901
79745147,Diesel,8998.0,2013.0,2012.0,Manuell,Renault Kangoo II 1.5 dCi FAP (110hk) Style 5 -13,74900.0,kangoo,78181.329962,0.958029


In [35]:
index = df_kangoo['header'] == 'Renault Kangoo 1,5 (6 växlad) -08'
df_kangoo.loc[index]

,Bränsle,Miltal,Modellår,Tillverkningsår,Växellåda,header,price,car type,predicted price,price rating
80790572,Diesel,15749.5,2008.0,2008.0,Manuell,"Renault Kangoo 1,5 (6 växlad) -08",45000.0,kangoo,30125.635766,1.493744


In [36]:
index = df_kangoo['header'] == 'Renault Kangoo 1.6|5-Sits|6800Mil|Fullservad -09'
df_kangoo.loc[index]



,Bränsle,Miltal,Modellår,Tillverkningsår,Växellåda,header,price,car type,predicted price,price rating
80603429,Bensin,6800.0,2009.0,2008.0,Manuell,Renault Kangoo 1.6|5-Sits|6800Mil|Fullservad -09,54900.0,kangoo,62411.169774,0.87965
